In [2]:
import os
import sys
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [3]:
path = os.getenv("ROOT_PATH")
sys.path.append(path)
print(path)


/Users/monic/Desktop/Master_Thesis/empirical


In [4]:
tickers= pd.read_excel(f"{path}/raw_data/Tickers.xlsx",header=0)


In [5]:
tickers.head()


,Company Name,Security Symbol
0,TRATON SE,8TRA.ST
1,AAK-AB,AAK.ST
2,ABB Ltd,ABB.ST
3,Abliva-AB,ABLI.ST
4,AcadeMedia-AB,ACAD.ST


In [6]:
tickers_list=tickers['Security Symbol'].to_list()


In [9]:
data = yf.download(tickers_list, start="2013-01-01")


/Users/monic/.pyenv/versions/3.10.6/envs/empirical/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[                       0%%                      ]/Users/monic/.pyenv/versions/3.10.6/envs/empirical/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/Users/monic/.pyenv/versions/3.10.6/envs/empirical/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[                       1%%  

In [10]:
data.head()


Price      Adj Close                                                   \
Ticker       8TRA.ST     AAK.ST      ABB.ST    ABLI.ST ACAD.ST ACE.ST   
Date                                                                    
2013-01-02       NaN  39.280941  132.599045  14.265562     NaN    NaN   
2013-01-03       NaN  39.280941  131.158768  14.188866     NaN    NaN   
2013-01-04       NaN  39.565582  131.926926  13.882079     NaN    NaN   
2013-01-07       NaN  39.779068  130.774704  13.805383     NaN    NaN   
2013-01-08       NaN  39.280941  131.350830  14.112169     NaN    NaN   

Price                                                 ...     Volume  \
Ticker     ACRI-A.ST ACRI-B.ST    ACTI.ST  ADDT-B.ST  ... VPLAY-B.ST   
Date                                                  ...              
2013-01-02       NaN       NaN  39.593727  11.440966  ...        NaN   
2013-01-03       NaN       NaN  39.942570  11.410128  ...        NaN   
2013-01-04       NaN       NaN  39.942570  11.502643  ...        NaN   
2013-01-07       NaN       NaN  39.768147  11.502643  ...        NaN   
2013-01-08       NaN       NaN  38.896042  11.718509  ...        NaN   

Price                                                                        \
Ticker     WALL-B.ST WBGR-B.ST WIHL.ST WISE.ST WTW-A.ST XANO-B.ST XBRANE.ST   
Date                                                                          
2013-01-02    329316       NaN  216712  3273.0      NaN      5632       NaN   
2013-01-03    185792       NaN  107776  2280.0      NaN     17749       NaN   
2013-01-04    160304       NaN  215020  3305.0      NaN     13726       NaN   
2013-01-07    226744       NaN  170032  9196.0      NaN     18712       NaN   
2013-01-08    495576       NaN  210944  2322.0      NaN     87030       NaN   

Price                          
Ticker     XSPRAY.ST XVIVO.ST  
Date                           
2013-01-02       NaN    37698  
2013-01-03       NaN    24970  
2013-01-04       NaN    37145  
2013-01-07       NaN    30260  
2013-01-08       NaN    23163  

[5 rows x 2358 columns]

In [11]:
data.to_csv(f"{path}/raw_data/raw_data.csv")
